In [1]:
from skimage.morphology import erosion, dilation
import scipy.ndimage
import numpy as np
import cv2
import sys
import random

In [2]:
def normalize_image(image, min_val=0, max_val=1):
    """
    Converts image values to range <min_val, max_val>.
    
    :param image: the image to convert
    :param min_val: the lower bound of the range
    :param max_val: the upper bound of the range
    
    :return: converted image
    """
    values_num = max_val - min_val
    
    img_min = min(image.flatten())
    img_max = max(image.flatten())

    if img_min != img_max:
        return (image - img_min)/(img_max - img_min) * values_num + min_val
    else:
        return (image - img_min) * values_num + min_val

In [3]:
def hu_to_pixels(image, rescale_intercept, rescale_slope, window_center, window_width):
    """
    Corrects the HU values by the rescale intercept, rescale slope and window, and converts them the to pixel values.
    
    :param image: the image to correct
    :param rescale_intercept: the rescale intercept for the image
    :param rescale_slope: the rescale slope for the image
    :param window_center: the window center / center level of the conversion
    :param window_width: the window width of the conversion
    
    :return: corrected image
    """
    corrected_hu = image * rescale_slope + rescale_intercept
    corrected_hu[corrected_hu <= -1000] = -1000
    
    windowed_image = np.array(corrected_hu)
    windowed_image[windowed_image < window_center - window_width/2] = window_center - window_width/2
    windowed_image[windowed_image > window_center + window_width/2] = window_center + window_width/2
    
    img_min = min(windowed_image.flatten())
    img_max = max(windowed_image.flatten())
    
    return normalize_image(windowed_image, 0, 225)

In [4]:
def get_brain_mask(image):
    """
    Generates mask of the brain.
    
    :param image: the image of brain
    
    :return: the mask of the brain
    """
    # do segmentation
    filtered = cv2.GaussianBlur(image, (5, 5), 10)
    _, seg = cv2.threshold(filtered, 20, 255, cv2.THRESH_BINARY)

    seg = erosion(seg, np.ones([5, 5]))
    seg = dilation(seg, np.ones([8, 8]))
    
    # find all contours
    contours, _  = cv2.findContours(seg.astype(np.uint8), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    # sort contours from biggest to smallest and find biggest solid and round contour - avoid selecting bigger elements in background than head slice
    contours.sort(key=cv2.contourArea, reverse=True)
    contour_biggest = None
    for cont in contours:
        hull = cv2.convexHull(cont, None, True, True)
        cont_area = cv2.contourArea(cont)
        hull_area = cv2.contourArea(hull)
        _,radius = cv2.minEnclosingCircle(cont)
        
        if cont_area / hull_area > 0.5 and (4 * cont_area) / (np.pi * 4 * radius**2) > 0.3:
            contour_biggest = hull
            return cv2.drawContours(np.zeros(image.shape, dtype=np.uint8), [contour_biggest], -1, color=(255, 255, 255), thickness=cv2.FILLED)
                                    
    return np.zeros(image.shape, dtype=np.uint8)

In [5]:
def segment_brain(image):
    """
    Segments brain from the CT image.
    
    :param image: the image for segmentation
    
    :return: the image with segmented brain
    """
    mask = get_brain_mask(image)
    return cv2.bitwise_and(image, image, mask = mask)

In [6]:
def crop_brain_area(image): 
    """
    Crops the brain by its bounding rectangle.
    
    :param image: the image of brain
    
    :return: the cropped image
    """
    mask = get_brain_mask(image)
    contours, _  = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    if len(contours) == 0:
        return np.zeros(image.shape, dtype=np.uint8)
    
    tl_x, tl_y, w, h = cv2.boundingRect(contours[0])
    return image[tl_y:tl_y+h, tl_x:tl_x+w]

In [7]:
def apply_new_spacing(image, old_spacing, new_spacing=[1, 1]):
    """
    Generates image with new pixel spacing.
    
    :param image: the image of brain
    :param old_spacing: the original pixel spacing
    :param new_spacing: the desired pixel spacing
    
    :return: the mask of the brain
    """
    raw_resize_factor = old_spacing / new_spacing
    new_shape = np.round(image.shape * raw_resize_factor)
    resize_factor = new_shape / image.shape
    return scipy.ndimage.interpolation.zoom(image, resize_factor, prefilter=False)

In [8]:
def crop_or_reshape(image, shape):
    """
    Crops image if its larger than the shape or otherwise pads it with zeros.
    
    :param image: the image to reshape
    :param shape: the new shape of the image
    
    :return: the reshaped image
    """
    reshaped_image = np.zeros(shape)
    
    height, width = image.shape
    
    if height > shape[1]:
        diff = height - shape[1]
        image = image[int(np.ceil(diff/2)):int(height-np.ceil(diff/2)), :]
        height = image.shape[0]
    
    if width > shape[0]:
        diff = width - shape[0]
        image = image[:, int(np.ceil(diff/2)):int(width-np.ceil(diff/2))]
        width = image.shape[1]
    
    pad_left = int((shape[0] - width)/2)
    pad_top = int((shape[1] - height)/2)

    reshaped_image[pad_top:pad_top + height,pad_left:pad_left + width] = image
    return reshaped_image

In [9]:
def clahe(image):
    """
    Applies CLAHE to an image.
    
    :param image: the image to apply CLAHE
    
    :return: the image after CLAHE
    """
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(10,10))
    return clahe.apply(image.astype('uint8'))

In [10]:
def to_3_channels(channel1, channel2=None, channel3=None):
    """
    Converts one-channel array to three-channel. If only one channel isprovided, it fills all three channels with the given one.
    
    :param channel1: the first channel
    :param channel2: the second channel
    :param channel3: the three channel
    
    :return: a three-channel array
    """
    if channel2 is None:
        channel2 = channel1
    if channel3 is None:
        channel3 = channel1
    
    image = np.empty((channel1.shape[0], channel1.shape[1], 3))
    image[:,:,0] = channel1
    image[:,:,1] = channel2
    image[:,:,2] = channel3
    
    return image

In [11]:
class DataAugmentation:
    """
    A class for data augmentation.
    """
    def __init__(self, seed, max_angle=45, h_max_percent=5, v_max_percent=5, zoom_range=0.1):
        """
        :param seed: the seed for random function
        :param max_angle: the maximum angle for rotation
        :param h_max_percent: the maximum percentage of horizontal shift
        :param v_max_percent: the maximum percentage of vertical shift
        :param zoom_range: the range for zooming
        """
        self.max_angle = max_angle
        self.h_max_percent = h_max_percent
        self.v_max_percent = v_max_percent
        self.zoom_range = zoom_range
        random.seed(seed)
        
    def rotate_image(self, image, seed=123, angle=None):
        """
        Rotates an image by an angle. If the angle is not provided, a random angle within the self.max_angle range will be used.
        
        :param image: the image to rotate
        :param seed: the seed for random function
        :param angle: the angle of the rotation
        
        :return: the rotated image
        """
        if angle is None:
            random.seed(seed)
            angle = random.randint(-self.max_angle, self.max_angle)

        height, width = image.shape[:2]
        center_x, center_y = (width // 2, height // 2)

        M = cv2.getRotationMatrix2D((center_x, center_y), angle, 1.0)
        return cv2.warpAffine(image, M, (width, height))
    
    def translate_image(self, image, seed=123, horizontal=None, vertical=None):
        """
        Translates an image horizontally and vertically. If the distances is not provided, a random distances will be used.
        
        :param image: the image to translate
        :param seed: the seed for random function
        :param horizontal: the horizontal shift distance
        :param vertical: the vertical shift distance
        
        :return: the translated image
        """
        random.seed(seed)

        height, width = image.shape[:2]
        if horizontal is None:
            horizontal = random.randint(- int(width/100 * self.h_max_percent), int(width/100 * self.h_max_percent))
        if vertical is None:
            vertical = random.randint(- int(height/100 * self.v_max_percent), int(height/100 * self.v_max_percent))

        M = np.float32([[1, 0, horizontal],[0, 1, vertical]])
        return cv2.warpAffine(image, M, (width, height))
    
    def zoom_image(self, image, seed=123, zoom=None):
        """
        Zooms an image. Id the zoom value is not provided, a random value within the self.zoom_range will be used.
        
        :param image: the image to zoom
        :param seed: the seed for random function
        :param zoom: the zoom value to be applied
        
        :return: the zoomed image
        """
        random.seed(seed)

        height, width = image.shape[:2]
        final_image = np.zeros(image.shape)
        if zoom is None:
            zoom = random.uniform(-self.zoom_range, self.zoom_range) + 1

        if zoom < 1:
            zoomed_image = cv2.resize(image,(int(zoom * width), int(zoom * height)), interpolation = cv2.INTER_AREA)
            pad_top, pad_left = ((height - int(zoom * height))//2, (width - int(zoom * width))//2)
            final_image[pad_top:pad_top + int(zoom * height),pad_left:pad_left + int(zoom * width)] = zoomed_image
        elif zoom > 1:
            zoomed_image = cv2.resize(image,(int(zoom * width), int(zoom * height)), interpolation = cv2.INTER_LINEAR)
            pad_top, pad_left = ((int(zoom * height) - height)//2, (int(zoom * width) - width)//2)
            final_image = zoomed_image[pad_top:pad_top + height,pad_left:pad_left + width]
        else:
            return image
        return final_image
    
    def add_noise(self, image, seed=123, max_value=10):
        """
        Adds random noise to an image.
        
        :param image: the image to add noise
        :param seed: the seed for random function
        :param max_value: the max value of the range within the noise is generated
        
        :return: the image with added noise
        """
        random.seed(seed)
        random_values = np.random.rand(image[:,:,0].shape[0], image[:,:,0].shape[1]) * max_value
        noise_image = image.copy()
        noise_image[:,:,0] += random_values.astype('uint8')
        noise_image[:,:,1] += random_values.astype('uint8')
        noise_image[:,:,2] += random_values.astype('uint8')
        return noise_image
            
    def random_augment(self, image, noise=False):
        """
        Performs an augmentation with random values using rotation, translation, zoom, and optionally adding random noise.
        
        :param image: the image to augment
        :param max_value: true of false - whether to use noise
        
        :return: the augmented image
        """
        seed = random.randint(1, sys.maxsize)
        rotated = self.rotate_image(image, seed)
        translated = self.translate_image(rotated, seed)
        zoomed = self.zoom_image(translated, seed)
        if not noise:
            return zoomed
        return self.add_noise(zoomed, seed)